In [29]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

print("📊 Naive Forecast: Umsatz vom letzten gleichen Wochentag")
print("="*60)

# 1. Daten laden und vorbereiten
print("📋 Lade Trainingsdaten...")
df = pd.read_csv('/workspaces/bakery_sales_prediction/5_Datasets/bakery_training_dataset.csv')

# Datum konvertieren
df['Datum'] = pd.to_datetime(df['Datum'])
print(f"Datensatz geladen: {len(df)} Zeilen von {df['Datum'].min()} bis {df['Datum'].max()}")
print(f"Warengruppen: {sorted(df['Warengruppe_Name'].unique())}")
print(f"Spalten: {list(df.columns)}")

# Wochentag hinzufügen für bessere Übersicht
df['Wochentag'] = df['Datum'].dt.day_name()
print(f"Datenbereiche nach Wochentag:")
for wochentag in ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']:
    count = len(df[df['Wochentag'] == wochentag])
    print(f"  {wochentag}: {count} Einträge")

📊 Naive Forecast: Umsatz vom letzten gleichen Wochentag
📋 Lade Trainingsdaten...
Datensatz geladen: 9334 Zeilen von 2013-07-01 00:00:00 bis 2018-07-31 00:00:00
Warengruppen: ['Brot', 'Brötchen', 'Croissant', 'Konditorei', 'Kuchen', 'Saisonbrot']
Spalten: ['Datum', 'Jahr', 'Monat', 'Tag', 'Wochentag', 'Wochentag_Nr', 'Warengruppe', 'Warengruppe_Name', 'Temperatur', 'Windgeschwindigkeit', 'Bewoelkung', 'Wettercode_fehlt', 'ist_feiertag', 'Jahreszeit', 'ist_kiwo', 'Umsatz', 'Warengruppe_Brot', 'Warengruppe_Brötchen', 'Warengruppe_Croissant', 'Warengruppe_Konditorei', 'Warengruppe_Kuchen', 'Warengruppe_Saisonbrot', 'Jahreszeit_Winter', 'Jahreszeit_Frühling', 'Jahreszeit_Sommer', 'Jahreszeit_Herbst', 'Wochentag_Nr.1', 'Wochentag_Monday', 'Wochentag_Tuesday', 'Wochentag_Wednesday', 'Wochentag_Thursday', 'Wochentag_Friday', 'Wochentag_Saturday', 'Wochentag_Sunday', 'Wettercode']
Datenbereiche nach Wochentag:
  Monday: 1324 Einträge
  Tuesday: 1345 Einträge
  Wednesday: 1342 Einträge
  Thursda

In [30]:
# 2. Warengruppen-Mapping für Sample Submission
warengruppe_mapping = {
    1: 'Brot',
    2: 'Brötchen', 
    3: 'Croissant',
    4: 'Konditorei',
    5: 'Kuchen',
    6: 'Saisonbrot'
}

def parse_sample_id(sample_id):
    """Parst Sample ID im Format YYMMDDW zu Datum und Warengruppe"""
    # Konvertiere zu Integer falls Float
    sample_int = int(float(sample_id))
    sample_str = str(sample_int)
    
    if len(sample_str) != 7:
        raise ValueError(f"Invalid sample ID format: {sample_id}")
    
    year = int('20' + sample_str[:2])
    month = int(sample_str[2:4])
    day = int(sample_str[4:6])
    warengruppe_id = int(sample_str[6])
    
    datum = pd.to_datetime(f'{year}-{month:02d}-{day:02d}')
    warengruppe_name = warengruppe_mapping.get(warengruppe_id, f'Unknown_{warengruppe_id}')
    
    return datum, warengruppe_name

# Test der Parsing-Funktion
test_id = 1808011
test_datum, test_warengruppe = parse_sample_id(test_id)
print(f"Test ID {test_id}: Datum={test_datum.date()}, Warengruppe={test_warengruppe}, Wochentag={test_datum.strftime('%A')}")

Test ID 1808011: Datum=2018-08-01, Warengruppe=Brot, Wochentag=Wednesday


In [31]:
# 3. Datensplit: Historische Daten bis 31.07.2018
cutoff_date = pd.to_datetime('2018-07-31')
historical_data = df[df['Datum'] <= cutoff_date].copy()

print(f"Historische Daten: {len(historical_data)} Zeilen bis {cutoff_date.date()}")
print(f"Letzte verfügbare Daten: {historical_data['Datum'].max().date()}")

# Erstelle Lookup-Dictionary für schnelle Suche: (Datum, Warengruppe) -> Umsatz
umsatz_lookup = {}
for _, row in historical_data.iterrows():
    key = (row['Datum'].date(), row['Warengruppe_Name'])
    umsatz_lookup[key] = row['Umsatz']

print(f"Lookup-Dictionary erstellt mit {len(umsatz_lookup)} Einträgen")

# Test des Lookups
test_date = pd.to_datetime('2018-07-25').date()  # Mittwoch vor dem 01.08.2018
test_warengruppe = 'Brot'
test_key = (test_date, test_warengruppe)
if test_key in umsatz_lookup:
    print(f"Test Lookup für {test_date} - {test_warengruppe}: {umsatz_lookup[test_key]:.2f}€")
else:
    print(f"Test Lookup für {test_date} - {test_warengruppe}: Nicht gefunden")

Historische Daten: 9334 Zeilen bis 2018-07-31
Letzte verfügbare Daten: 2018-07-31
Lookup-Dictionary erstellt mit 9334 Einträgen
Test Lookup für 2018-07-25 - Brot: 133.60€


In [32]:
# 4. Naive Vorhersage-Funktion: Gleicher Wochentag der Vorwoche
def get_last_weekday_forecast(target_date, warengruppe_name, umsatz_lookup, max_lookback_weeks=4):
    """
    Sucht den Umsatz vom gleichen Wochentag der Vorwoche.
    Falls nicht verfügbar, geht schrittweise weitere Wochen zurück.
    """
    for weeks_back in range(1, max_lookback_weeks + 1):
        reference_date = (target_date - timedelta(weeks=weeks_back)).date()
        lookup_key = (reference_date, warengruppe_name)
        
        if lookup_key in umsatz_lookup:
            return umsatz_lookup[lookup_key], reference_date, weeks_back
    
    # Falls kein Wert gefunden wurde, gib 0 zurück
    return 0.0, None, None

# Test der Vorhersage-Funktion
test_target = pd.to_datetime('2018-08-01')  # Mittwoch
forecast, ref_date, weeks_back = get_last_weekday_forecast(
    test_target, 'Brot', umsatz_lookup
)
print(f"Test Forecast für {test_target.date()} ({test_target.strftime('%A')}) - Brot:")
print(f"  Vorhersage: {forecast:.2f}€")
print(f"  Referenzdatum: {ref_date} ({weeks_back} Wochen zurück)")

Test Forecast für 2018-08-01 (Wednesday) - Brot:
  Vorhersage: 133.60€
  Referenzdatum: 2018-07-25 (1 Wochen zurück)


In [33]:
# 5. Sample Submission laden und verarbeiten
print("📋 Lade Sample Submission...")
sample_submission = pd.read_csv('/workspaces/bakery_sales_prediction/5_Datasets/sample_submission.csv')
print(f"Sample Submission: {len(sample_submission)} Zeilen")
print(f"Beispiel IDs: {sample_submission['id'].head().tolist()}")

# Parse alle Sample IDs
parsed_data = []
for sample_id in sample_submission['id']:
    try:
        datum, warengruppe_name = parse_sample_id(sample_id)
        parsed_data.append({
            'id': sample_id,
            'Datum': datum,
            'Warengruppe_Name': warengruppe_name,
            'Wochentag': datum.strftime('%A')
        })
    except Exception as e:
        print(f"Fehler beim Parsen von ID {sample_id}: {e}")

parsed_df = pd.DataFrame(parsed_data)
print(f"Erfolgreich geparst: {len(parsed_df)} IDs")
print(f"Vorhersagezeitraum: {parsed_df['Datum'].min().date()} - {parsed_df['Datum'].max().date()}")
print(f"Warengruppen: {sorted(parsed_df['Warengruppe_Name'].unique())}")

# Verteilung nach Wochentagen
print("\nVerteilung nach Wochentagen:")
wochentag_counts = parsed_df['Wochentag'].value_counts()
for wochentag in ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']:
    if wochentag in wochentag_counts.index:
        print(f"  {wochentag}: {wochentag_counts[wochentag]} Vorhersagen")

📋 Lade Sample Submission...
Sample Submission: 1830 Zeilen
Beispiel IDs: [1808011, 1808021, 1808031, 1808041, 1808051]
Erfolgreich geparst: 1830 IDs
Vorhersagezeitraum: 2018-08-01 - 2019-07-30
Warengruppen: ['Brot', 'Brötchen', 'Croissant', 'Konditorei', 'Kuchen', 'Saisonbrot']

Verteilung nach Wochentagen:
  Monday: 267 Vorhersagen
  Tuesday: 257 Vorhersagen
  Wednesday: 237 Vorhersagen
  Thursday: 269 Vorhersagen
  Friday: 264 Vorhersagen
  Saturday: 268 Vorhersagen
  Sunday: 268 Vorhersagen


In [34]:
# 6. Generiere Vorhersagen für alle Sample IDs
print("🔮 Generiere Vorhersagen basierend auf letztem gleichen Wochentag...")

predictions = []
stats = {
    'successful': 0,
    'fallback_used': 0,
    'no_data_found': 0,
    'weeks_back_distribution': {}
}

for idx, row in parsed_df.iterrows():
    if idx % 200 == 0:
        print(f"Fortschritt: {idx}/{len(parsed_df)}")
    
    forecast, ref_date, weeks_back = get_last_weekday_forecast(
        target_date=row['Datum'],
        warengruppe_name=row['Warengruppe_Name'],
        umsatz_lookup=umsatz_lookup,
        max_lookback_weeks=8  # Erweitert auf 8 Wochen für mehr Robustheit
    )
    
    predictions.append({
        'id': row['id'],
        'Umsatz': forecast
    })
    
    # Statistiken sammeln
    if forecast > 0:
        stats['successful'] += 1
        if weeks_back > 1:
            stats['fallback_used'] += 1
        
        # Verteilung der Wochen zurück
        if weeks_back not in stats['weeks_back_distribution']:
            stats['weeks_back_distribution'][weeks_back] = 0
        stats['weeks_back_distribution'][weeks_back] += 1
    else:
        stats['no_data_found'] += 1

# Erstelle Prediction DataFrame
prediction_df = pd.DataFrame(predictions)
print(f"\nVorhersagen generiert: {len(prediction_df)} Zeilen")
print(f"Durchschnittlicher Umsatz: {prediction_df['Umsatz'].mean():.2f}€")
print(f"Min/Max Umsatz: {prediction_df['Umsatz'].min():.2f}€ / {prediction_df['Umsatz'].max():.2f}€")

print(f"\nStatistiken:")
print(f"  Erfolgreiche Vorhersagen: {stats['successful']}")
print(f"  Fallback verwendet (>1 Woche): {stats['fallback_used']}")
print(f"  Keine Daten gefunden: {stats['no_data_found']}")
print(f"  Verteilung Wochen zurück: {stats['weeks_back_distribution']}")

🔮 Generiere Vorhersagen basierend auf letztem gleichen Wochentag...
Fortschritt: 0/1830
Fortschritt: 200/1830
Fortschritt: 400/1830
Fortschritt: 600/1830
Fortschritt: 800/1830
Fortschritt: 1000/1830
Fortschritt: 1200/1830
Fortschritt: 1400/1830
Fortschritt: 1600/1830
Fortschritt: 1800/1830

Vorhersagen generiert: 1830 Zeilen
Durchschnittlicher Umsatz: 47.16€
Min/Max Umsatz: 0.00€ / 721.82€

Statistiken:
  Erfolgreiche Vorhersagen: 280
  Fallback verwendet (>1 Woche): 245
  Keine Daten gefunden: 1550
  Verteilung Wochen zurück: {1: 35, 2: 35, 3: 35, 4: 35, 5: 35, 6: 35, 7: 35, 8: 35}


In [ ]:
# 7. Speichere finale Vorhersagen
print("💾 Speichere finale Vorhersagen...")

# Stelle sicher, dass Reihenfolge identisch zur Sample Submission ist
final_predictions = sample_submission[['id']].merge(prediction_df, on='id', how='left')

# Prüfe auf fehlende Vorhersagen
missing_predictions = final_predictions['Umsatz'].isna().sum()
if missing_predictions > 0:
    print(f"⚠️ Warnung: {missing_predictions} fehlende Vorhersagen gefunden!")
    final_predictions['Umsatz'].fillna(0, inplace=True)

# Speichere CSV
output_path = '/workspaces/bakery_sales_prediction/2_BaselineModel/prediction_naive_forecasting_baseline.csv'
final_predictions.to_csv(output_path, index=False)

print(f"✅ Vorhersagen gespeichert: {output_path}")
print(f"Anzahl Zeilen: {len(final_predictions)}")
print(f"Spalten: {list(final_predictions.columns)}")

# Zeige erste Vorhersagen
print("\nErste 10 Vorhersagen:")
display_df = final_predictions.head(10).copy()
for idx, row in display_df.iterrows():
    try:
        datum, warengruppe = parse_sample_id(row['id'])
        wochentag = datum.strftime('%A')
        print(f"ID {int(row['id'])}: {datum.date()} ({wochentag}) - {warengruppe} - {row['Umsatz']:.2f}€")
    except Exception as e:
        print(f"Fehler bei ID {row['id']}: {e}")

💾 Speichere finale Vorhersagen...
✅ Vorhersagen gespeichert: /workspaces/bakery_sales_prediction/2_BaselineModel/prediction.csv
Anzahl Zeilen: 1830
Spalten: ['id', 'Umsatz']

Erste 10 Vorhersagen:
ID 1808011: 2018-08-01 (Wednesday) - Brot - 133.60€
ID 1808021: 2018-08-02 (Thursday) - Brot - 202.54€
ID 1808031: 2018-08-03 (Friday) - Brot - 198.23€
ID 1808041: 2018-08-04 (Saturday) - Brot - 214.47€
ID 1808051: 2018-08-05 (Sunday) - Brot - 129.80€
ID 1808061: 2018-08-06 (Monday) - Brot - 148.54€
ID 1808071: 2018-08-07 (Tuesday) - Brot - 123.08€
ID 1808081: 2018-08-08 (Wednesday) - Brot - 133.60€
ID 1808091: 2018-08-09 (Thursday) - Brot - 202.54€
ID 1808101: 2018-08-10 (Friday) - Brot - 198.23€


In [36]:
# 8. Detaillierte Analyse der Vorhersagen
print("📈 Detaillierte Analyse der Vorhersagen:")
print("="*60)

# Analyse pro Warengruppe
warengruppe_analysis = parsed_df.merge(prediction_df, on='id')
warengruppe_stats = warengruppe_analysis.groupby('Warengruppe_Name')['Umsatz'].agg([
    'count', 'mean', 'std', 'min', 'max'
]).round(2)

print("Vorhersagen pro Warengruppe:")
print(warengruppe_stats)

# Analyse pro Wochentag
print("\nVorhersagen pro Wochentag:")
wochentag_stats = warengruppe_analysis.groupby('Wochentag')['Umsatz'].agg([
    'count', 'mean', 'std', 'min', 'max'
]).round(2)
# Sortiere nach Wochentag-Reihenfolge
wochentag_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
wochentag_stats = wochentag_stats.reindex([day for day in wochentag_order if day in wochentag_stats.index])
print(wochentag_stats)

# Analyse der Null-Vorhersagen
null_predictions = warengruppe_analysis[warengruppe_analysis['Umsatz'] == 0]
if len(null_predictions) > 0:
    print(f"\n⚠️ Null-Vorhersagen: {len(null_predictions)} von {len(warengruppe_analysis)}")
    null_by_warengruppe = null_predictions.groupby('Warengruppe_Name').size()
    print("Null-Vorhersagen pro Warengruppe:")
    for warengruppe, count in null_by_warengruppe.items():
        print(f"  {warengruppe}: {count}")

print(f"\nModell-Zusammenfassung:")
print(f"Modell: Naive Forecast - Gleicher Wochentag der Vorwoche")
print(f"Historischer Datenzeitraum: {historical_data['Datum'].min().date()} - {historical_data['Datum'].max().date()}")
print(f"Vorhersagezeitraum: {parsed_df['Datum'].min().date()} - {parsed_df['Datum'].max().date()}")
print(f"Anzahl Vorhersagen: {len(final_predictions)}")
print(f"Erfolgsrate: {(stats['successful']/len(parsed_df)*100):.1f}%")
print(f"Durchschnittlicher vorhergesagter Umsatz: {final_predictions['Umsatz'].mean():.2f}€")

📈 Detaillierte Analyse der Vorhersagen:
Vorhersagen pro Warengruppe:
                  count    mean     std  min     max
Warengruppe_Name                                    
Brot                355   25.92   61.69  0.0  214.47
Brötchen            355  103.62  240.52  0.0  721.82
Croissant           355   55.23  128.87  0.0  433.82
Konditorei          354   11.78   28.74  0.0  128.95
Kuchen              355   46.58  108.70  0.0  335.82
Saisonbrot           56    0.00    0.00  0.0    0.00

Vorhersagen pro Wochentag:
           count   mean     std  min     max
Wochentag                                   
Monday       267  45.26  130.47  0.0  616.36
Tuesday      257  40.89  119.66  0.0  586.08
Wednesday    237  48.17  136.09  0.0  651.75
Thursday     269  46.59  132.64  0.0  628.29
Friday       264  47.66  138.17  0.0  677.42
Saturday     268  49.16  145.22  0.0  716.58
Sunday       268  52.25  151.30  0.0  721.82

⚠️ Null-Vorhersagen: 1550 von 1830
Null-Vorhersagen pro Warengruppe:
  Br

In [37]:
# 9. Validierung: Teste das Modell auf einem bekannten Zeitraum
print("📊 Validierung des Modells...")

# Verwende Juli 2018 als Validierungsmonat (noch in historischen Daten enthalten)
val_start = pd.to_datetime('2018-07-01')
val_end = pd.to_datetime('2018-07-31')

# Erstelle Validierungsdaten
val_data = historical_data[
    (historical_data['Datum'] >= val_start) & 
    (historical_data['Datum'] <= val_end)
].copy()

# Erstelle historische Daten nur bis Juni 2018 für Validierung
val_historical = historical_data[historical_data['Datum'] < val_start].copy()
val_umsatz_lookup = {}
for _, row in val_historical.iterrows():
    key = (row['Datum'].date(), row['Warengruppe_Name'])
    val_umsatz_lookup[key] = row['Umsatz']

print(f"Validierungsdaten: {len(val_data)} Einträge im Juli 2018")
print(f"Validierungs-Lookup: {len(val_umsatz_lookup)} historische Einträge bis Juni 2018")

# Generiere Vorhersagen für Validierungsmonat
val_predictions = []
val_actuals = []

for _, row in val_data.iterrows():
    # Vorhersage mit gleichem Modell
    forecast, _, _ = get_last_weekday_forecast(
        target_date=row['Datum'],
        warengruppe_name=row['Warengruppe_Name'],
        umsatz_lookup=val_umsatz_lookup,
        max_lookback_weeks=8
    )
    
    val_predictions.append(forecast)
    val_actuals.append(row['Umsatz'])

val_predictions = np.array(val_predictions)
val_actuals = np.array(val_actuals)

# Berechne Validierungsmetriken
mae = np.mean(np.abs(val_predictions - val_actuals))
mse = np.mean((val_predictions - val_actuals) ** 2)
rmse = np.sqrt(mse)

# R²
ss_res = np.sum((val_actuals - val_predictions) ** 2)
ss_tot = np.sum((val_actuals - np.mean(val_actuals)) ** 2)
r2 = 1 - (ss_res / ss_tot)

print(f"\nValidierungs-Metriken (Juli 2018):")
print(f"MAE: {mae:.2f}€")
print(f"RMSE: {rmse:.2f}€")
print(f"R²: {r2:.4f}")
print(f"Anzahl Validierungsvorhersagen: {len(val_predictions)}")
print(f"Durchschnittlicher tatsächlicher Umsatz: {val_actuals.mean():.2f}€")
print(f"Durchschnittlicher vorhergesagter Umsatz: {val_predictions.mean():.2f}€")

📊 Validierung des Modells...
Validierungsdaten: 155 Einträge im Juli 2018
Validierungs-Lookup: 9179 historische Einträge bis Juni 2018

Validierungs-Metriken (Juli 2018):
MAE: 78.70€
RMSE: 107.22€
R²: 0.6497
Anzahl Validierungsvorhersagen: 155
Durchschnittlicher tatsächlicher Umsatz: 284.88€
Durchschnittlicher vorhergesagter Umsatz: 223.24€
